# Data Cleaning and Resampling for Energy Consumption Data
This Jupyter Notebook code block focuses on cleaning and resampling energy consumption data from the different CSV files given by paul.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df_1 = pd.read_csv("2022-12-01_17 11_influxdb_data.csv", header=3)
df_2 = pd.read_csv("geo-energy-data-2023-02-01_cleaned.csv", header=3, low_memory=False)
df_new = pd.read_csv("geo-energy-data-2023-03-16.csv", header=3, low_memory=False)

# Combine all three dataframes
df = pd.concat([df_1, df_2, df_new], ignore_index=True)
df.shape
df.head()
df1 = df.drop(df.columns[[0, 1]], axis=1)
df2 = df1.rename(columns = 
               {'_start': 'start',
                '_stop': 'stop',
                '_time': 'tstp',
                '_value': 'energy',
                '_field': 'field',
                '_measurement': 'measurement'})
print(df2)

## Data Cleaning
After the DataFrames are combined, I started the cleaning process. The cleaned DataFrame is then preprocessed for the only useful columns such as timestamp and power which is saved as a CSV file called "data_minutes.csv".

In [2]:
print(df2)
df2_cleaned =df2[df2['field']=='val']
print(df.dtypes)
convert_dict = {'field': str, 'unit': str, 'start': str}
df3 = df2_cleaned.astype(convert_dict)
print(df3)

        field             measurement                           start  \
0         val               meterdata  2022-11-30T17:10:27.342968311Z   
1         val               meterdata  2022-11-30T17:10:27.342968311Z   
2         val               meterdata  2022-11-30T17:10:27.342968311Z   
3         val               meterdata  2022-11-30T17:10:27.342968311Z   
4         val               meterdata  2022-11-30T17:10:27.342968311Z   
...       ...                     ...                             ...   
1685984   val  meterdata_currentcosts  2023-01-15T16:08:13.209656531Z   
1685985   val  meterdata_currentcosts  2023-01-15T16:08:13.209656531Z   
1685986   val  meterdata_currentcosts  2023-01-15T16:08:13.209656531Z   
1685987   val  meterdata_currentcosts  2023-01-15T16:08:13.209656531Z   
1685988   val  meterdata_currentcosts  2023-01-15T16:08:13.209656531Z   

                                   stop                  tstp   energy result  \
0        2022-12-01T17:10:27.342968311Z  2

In [3]:
df2_cleaned.isna().sum()
df_3 = df2_cleaned.dropna(subset=['tstp', 'energy'])
print(df_3)
df_3.isna().sum()


        field             measurement                           start  \
0         val               meterdata  2022-11-30T17:10:27.342968311Z   
1         val               meterdata  2022-11-30T17:10:27.342968311Z   
2         val               meterdata  2022-11-30T17:10:27.342968311Z   
3         val               meterdata  2022-11-30T17:10:27.342968311Z   
4         val               meterdata  2022-11-30T17:10:27.342968311Z   
...       ...                     ...                             ...   
1685984   val  meterdata_currentcosts  2023-01-15T16:08:13.209656531Z   
1685985   val  meterdata_currentcosts  2023-01-15T16:08:13.209656531Z   
1685986   val  meterdata_currentcosts  2023-01-15T16:08:13.209656531Z   
1685987   val  meterdata_currentcosts  2023-01-15T16:08:13.209656531Z   
1685988   val  meterdata_currentcosts  2023-01-15T16:08:13.209656531Z   

                                   stop                  tstp   energy result  \
0        2022-12-01T17:10:27.342968311Z  2

field                0
measurement          0
start                0
stop                 0
tstp                 0
energy               0
result         1685972
source               0
table                0
type                 0
unit             17186
dtype: int64

In [4]:
df4 = df3[df3["type"]!="GAS_ENERGY"]
print(df4)

        field             measurement                           start  \
0         val               meterdata  2022-11-30T17:10:27.342968311Z   
1         val               meterdata  2022-11-30T17:10:27.342968311Z   
2         val               meterdata  2022-11-30T17:10:27.342968311Z   
3         val               meterdata  2022-11-30T17:10:27.342968311Z   
4         val               meterdata  2022-11-30T17:10:27.342968311Z   
...       ...                     ...                             ...   
1685984   val  meterdata_currentcosts  2023-01-15T16:08:13.209656531Z   
1685985   val  meterdata_currentcosts  2023-01-15T16:08:13.209656531Z   
1685986   val  meterdata_currentcosts  2023-01-15T16:08:13.209656531Z   
1685987   val  meterdata_currentcosts  2023-01-15T16:08:13.209656531Z   
1685988   val  meterdata_currentcosts  2023-01-15T16:08:13.209656531Z   

                                   stop                  tstp   energy result  \
0        2022-12-01T17:10:27.342968311Z  2

In [5]:
columns_to_drop = ["table", "start", "stop", "field", "measurement", "source", "unit", "type", "Unnamed: 12","result"]
columns_to_drop = [col for col in columns_to_drop if col in df4.columns]

df5 = df4.drop(columns=columns_to_drop)
print(df5)

                         tstp   energy
0        2022-11-30T17:12:00Z  515.333
1        2022-11-30T17:16:00Z  228.591
2        2022-11-30T17:20:00Z  184.708
3        2022-11-30T17:24:00Z   349.04
4        2022-11-30T17:28:00Z  362.125
...                       ...      ...
1685984  2023-02-12T20:26:48Z  370.204
1685985  2023-02-12T20:31:48Z  371.433
1685986  2023-02-12T20:36:48Z  372.583
1685987  2023-02-12T20:41:48Z  373.614
1685988  2023-02-12T20:46:48Z  374.367

[927861 rows x 2 columns]


## Resampling
Resampling the DataFrame by aggregating data in 30-minute intervals. The energy values for each 30-minute interval are averaged. A new column "LCLid" is added to the DataFrame to relate to a London Dataset and then the resulting DataFrame is saved as a CSV file called "data_30_minutes.csv".

In [14]:
problem_rows = df5.loc[~pd.to_datetime(df5["tstp"], errors='coerce').notnull()]
print(problem_rows)
df6 = df5.drop(problem_rows.index)
print(df6)
convert_dict = {'energy': float}
df7 = df6.astype(convert_dict)
df7["tstp"]= pd.to_datetime(df7["tstp"])
print(df7)
df7.to_csv("data_minutes.csv", index=True)

Empty DataFrame
Columns: [tstp, energy]
Index: []
                         tstp   energy
0        2022-11-30T17:12:00Z  515.333
1        2022-11-30T17:16:00Z  228.591
2        2022-11-30T17:20:00Z  184.708
3        2022-11-30T17:24:00Z   349.04
4        2022-11-30T17:28:00Z  362.125
...                       ...      ...
1685984  2023-02-12T20:26:48Z  370.204
1685985  2023-02-12T20:31:48Z  371.433
1685986  2023-02-12T20:36:48Z  372.583
1685987  2023-02-12T20:41:48Z  373.614
1685988  2023-02-12T20:46:48Z  374.367

[927861 rows x 2 columns]
                             tstp      energy
0       2022-11-30 17:12:00+00:00  515.333333
1       2022-11-30 17:16:00+00:00  228.590909
2       2022-11-30 17:20:00+00:00  184.708333
3       2022-11-30 17:24:00+00:00  349.040000
4       2022-11-30 17:28:00+00:00  362.125000
...                           ...         ...
1685984 2023-02-12 20:26:48+00:00  370.204000
1685985 2023-02-12 20:31:48+00:00  371.433000
1685986 2023-02-12 20:36:48+00:00  372.58

In [7]:
df7["tstp"]= pd.to_datetime(df7["tstp"])
#using the mean of the 30 mins intervales
df7["tstp"] = df7["tstp"].dt.floor('30T')
# Resample the data based on the new column, taking the mean of the energy values
df8 = df7.groupby("tstp").mean()
print(df8)

                               energy
tstp                                 
2022-11-30 17:00:00+00:00  327.959515
2022-11-30 17:30:00+00:00  476.221791
2022-11-30 18:00:00+00:00  265.523489
2022-11-30 18:30:00+00:00  252.229837
2022-11-30 19:00:00+00:00  412.906994
...                               ...
2023-03-16 14:00:00+00:00  911.928525
2023-03-16 14:30:00+00:00  816.426538
2023-03-16 15:00:00+00:00  780.541752
2023-03-16 15:30:00+00:00  761.983622
2023-03-16 16:00:00+00:00  796.429589

[3530 rows x 1 columns]


In [8]:
df8["LCLid"] = "MAC000001"

In [9]:
print(df8)

                               energy      LCLid
tstp                                            
2022-11-30 17:00:00+00:00  327.959515  MAC000001
2022-11-30 17:30:00+00:00  476.221791  MAC000001
2022-11-30 18:00:00+00:00  265.523489  MAC000001
2022-11-30 18:30:00+00:00  252.229837  MAC000001
2022-11-30 19:00:00+00:00  412.906994  MAC000001
...                               ...        ...
2023-03-16 14:00:00+00:00  911.928525  MAC000001
2023-03-16 14:30:00+00:00  816.426538  MAC000001
2023-03-16 15:00:00+00:00  780.541752  MAC000001
2023-03-16 15:30:00+00:00  761.983622  MAC000001
2023-03-16 16:00:00+00:00  796.429589  MAC000001

[3530 rows x 2 columns]


In [10]:
df9 = df8.reset_index().set_index("LCLid")

In [11]:
print(df9)

                               tstp      energy
LCLid                                          
MAC000001 2022-11-30 17:00:00+00:00  327.959515
MAC000001 2022-11-30 17:30:00+00:00  476.221791
MAC000001 2022-11-30 18:00:00+00:00  265.523489
MAC000001 2022-11-30 18:30:00+00:00  252.229837
MAC000001 2022-11-30 19:00:00+00:00  412.906994
...                             ...         ...
MAC000001 2023-03-16 14:00:00+00:00  911.928525
MAC000001 2023-03-16 14:30:00+00:00  816.426538
MAC000001 2023-03-16 15:00:00+00:00  780.541752
MAC000001 2023-03-16 15:30:00+00:00  761.983622
MAC000001 2023-03-16 16:00:00+00:00  796.429589

[3530 rows x 2 columns]


In [12]:
df9.to_csv("data_30_minutes.csv", index=True)